In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from pymongo import MongoClient
client = MongoClient()

In [3]:
# Dealing with the headline

def get_mars_headline():
    url = "https://mars.nasa.gov/api/v1/news_items/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    resp = requests.get(url).json()
    first_item = resp.get('items')[0]
    return {"item_title": first_item.get('title'), 
            "item_desc": first_item.get('description')
           }

headlines_dict = get_mars_headline()

headlines_dict

{'item_title': 'After a Reset, Curiosity Is Operating Normally',
 'item_desc': 'Curiosity has returned to science operations and is once again exploring the clay unit. '}

In [4]:
def get_mars_facts_table():
    tables = pd.read_html("https://space-facts.com/mars/") # list of tables, even if there's 1
    table_we_want = tables[0]
    table_we_want.columns = ['Thing', "Metric"]
    formatted =  table_we_want.to_html(classes=["table-bordered", "table-striped", "table-hover"])
    return {"html_table_facts": formatted}

facts_table_dict = get_mars_facts_table()
facts_table_dict

{'html_table_facts': '<table border="1" class="dataframe table-bordered table-striped table-hover">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Thing</th>\n      <th>Metric</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th

In [32]:
# Insert into Mongo - but first merge our dictionaries

merged_dict = {**facts_table_dict, **headlines_dict}

In [33]:
db = client.test_mars
collection = db.test_collection
collection.insert_one(merged_dict)